# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.oncvpsp3.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397813246189                   -0.90    5.0   26.1ms
  2   -8.400251927236       -2.61       -1.74    1.0   41.0ms
  3   -8.400407474207       -3.81       -2.95    1.5   20.1ms
  4   -8.400428711779       -4.67       -2.99    3.0   24.1ms
  5   -8.400428822234       -6.96       -3.06    1.0   18.8ms
  6   -8.400429017986       -6.71       -4.61    1.0   18.7ms
  7   -8.400429023993       -8.22       -4.46    2.5   38.6ms
  8   -8.400429024396       -9.39       -5.24    1.0   19.3ms
  9   -8.400429024418      -10.64       -6.28    1.5   20.5ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397862365247                   -0.90           5.2    503ms
  2   -8.400385905432       -2.60       -1.79   0.80    2.0    503ms
  3   -8.400423698494       -4.42       -3.03   0.80    1.0    173ms
  4   -8.400428984324       -5.28       -3.41   0.80    2.5   20.4ms
  5   -8.400429021841       -7.43       -4.43   0.80    1.2   16.6ms
  6   -8.400429024378       -8.60       -5.44   0.80    2.2   19.6ms
  7   -8.400429024419      -10.38       -6.27   0.80    2.0   19.0ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.604593747251                   -1.06    2.91s
  2   -1.418072924455        0.31       -0.66    110ms
  3   -3.859324213125        0.39       -0.40   68.6ms
  4   -4.550784770540       -0.16       -0.49   39.9ms
  5   -6.682372622957        0.33       -0.53   39.1ms
  6   -7.477764944965       -0.10       -0.87   39.1ms
  7   -7.835289067803       -0.45       -1.32   47.7ms
  8   -8.166140891907       -0.48       -1.70   28.8ms
  9   -8.263723756402       -1.01       -1.84   28.8ms
 10   -8.327038718320       -1.20       -1.59   39.1ms
 11   -8.366514919488       -1.40       -1.79   43.0ms
 12   -8.380677535718       -1.85       -2.96   29.9ms
 13   -8.384781402749       -2.39       -2.93   29.1ms
 14   -8.389745400636       -2.30       -3.21   32.1ms
 15   -8.390922865211       -2.93       -3.15   29.0ms
 16   -8.392436654247       -2.82       -3.25   28.9ms
 17   -8.39

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397857091883                   -0.90    5.0   26.1ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400428854994                   -1.78    24.9s
  2   -8.400429024420       -6.77       -4.03    2.86s
  3   -8.400429024420      -14.27       -7.83    106ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.0189292052508307e-6
|ρ_newton - ρ_scfv| = 3.197711635440982e-7
|ρ_newton - ρ_dm|   = 0.019961842918314248
